# **Memory & Chain**

## **1.환경준비**

### (1) 라이브러리 Import

In [2]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [3]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

sk-proj-fNGRJcjczKaMAYUO9XV9T3BlbkFJ6VtsKrhyVUghaiFWcWce


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Memory**

### (1) 대화 기록 저장하기

In [4]:
# 메모리 선언하기(초기화)
memory = ConversationBufferMemory(return_messages=True)

# 저장
memory.save_context({"input": "안녕하세요!"},
                    {"output": "안녕하세요! 어떻게 도와드릴까요?"})
# input: human message, output: LLM의 답변

memory.save_context({"input": "메일을 써야하는데 도와줘"},
                    {"output": "누구에게 보내는 어떤 메일인가요?"})

# 현재 담겨 있는 메모리 내용 전체 확인
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕하세요!'),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?'),
  HumanMessage(content='메일을 써야하는데 도와줘'),
  AIMessage(content='누구에게 보내는 어떤 메일인가요?')]}

### (2) GPT와 대화 기록 저장하기

#### 1) 대화 준비

In [5]:
k = 3

# Chroma 데이터베이스 인스턴스
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)

# retriever 선언
retriever = database.as_retriever(search_kwargs={"k": k})

# ChatOpenAI 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")

# RetrievalQA 선언
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

#### 2) 대화 시도 및 기록

In [6]:
# 질문 답변1
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query)
result['result']

'생성형 AI 프로젝트는 보안 위협에 노출될 수 있는 다양한 영역이 있습니다. 예를 들어, 해커들이 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 등의 공격이 있을 수 있습니다. 또한, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 공격도 가능합니다. 이러한 보안 위협에 대비하기 위해서는 관리가 잘 이루어져야 하며, 모델을 테스트하고 평가하는 등의 조치가 필요할 수 있습니다.'

In [7]:
# 메모리 선언하기(초기화)
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 보안 위협에 노출될 수 있는 다양한 영역이 있습니다. 예를 들어, 해커들이 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 등의 공격이 있을 수 있습니다. 또한, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 공격도 가능합니다. 이러한 보안 위협에 대비하기 위해서는 관리가 잘 이루어져야 하며, 모델을 테스트하고 평가하는 등의 조치가 필요할 수 있습니다.')]}

#### 3) 이어지는 질문과 답변

In [8]:
# 질문 답변2
query = "훈련 데이터나 가중치를 오염시키는게 무슨 의미야?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 보안 위협에 노출될 수 있는 다양한 영역이 있습니다. 예를 들어, 해커들이 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 등의 공격이 있을 수 있습니다. 또한, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 공격도 가능합니다. 이러한 보안 위협에 대비하기 위해서는 관리가 잘 이루어져야 하며, 모델을 테스트하고 평가하는 등의 조치가 필요할 수 있습니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 악의적인 공격자가 AI 모델을 손상시키기 위해 해당 데이터나 모델 내의 가중치를 조작하거나 변조하는 것을 의미합니다. 이로 인해 모델이 잘못된 판단을 내리거나 부정확한 예측을 하게 되는 등 모델의 성능이 저하될 수 있습니다.')]}

In [9]:
# 질문 답변3
query = "이를 방지하기 위해 어떻게 해야 해?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 보안 위협에 노출될 수 있는 다양한 영역이 있습니다. 예를 들어, 해커들이 프로젝트의 개발팀에 침투하여 소프트웨어에 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 등의 공격이 있을 수 있습니다. 또한, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 공격도 가능합니다. 이러한 보안 위협에 대비하기 위해서는 관리가 잘 이루어져야 하며, 모델을 테스트하고 평가하는 등의 조치가 필요할 수 있습니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 악의적인 공격자가 AI 모델을 손상시키기 위해 해당 데이터나 모델 내의 가중치를 조작하거나 변조하는 것을 의미합니다. 이로 인해 모델이 잘못된 판단을 내리거나 부정확한 예측을 하게 되는 등 모델의 성능이 저하될 수 있습니다.'),
  HumanMessage(content='이를 방지하기 위해 어떻게 해야 해?'),
  AIMessage(content='악의적인 공격을 방지하기 위해서는 다음과 같은 조치를 취할 수 있습니다:\n1. 오픈소스 모델을 사용할 때에는 신중하게 검토하고, 신뢰할 수 있는 소스에서 다운로드해야 합니다.\n2. 보안 전문가나 데이터 과학자를 고용하여 오픈소스 AI 모델을 모니터링하고 취약점을 분석하도록 할 수 있습니다.\n3. 모델에 워터마크와 같은 추가적인 보안 기능을 적용하여 데이터의 무결성을 유지할 수 있습니다.\n4. 모델을 사용하는 시스템의 일부로서, 모델의 출력 결과를 다른 부분에서 사용하는 경우에는 시스템 전체를 고려하여 보안을 강화할 필요가 있습니다.\n5. 가능한 경우 모델을 테스트하고 평가하여 취약점을 발견하고 보호 장치를 마련하는 것이 중요합니다

* 훈련 데이터 오염을 방지하기 위한 대책을 물었으나, 일반적인 보안 위협 방지 대책을 이야기 함.
* 맥락을 유지하기 위해서 메시지의 내용을 프롬프트에 포함시켜야 함.
    * 이를 손쉽게 엮어주는 방법 **Chain**

## **3.Chain**

* 절차 다시 정리
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 이전 대화 내용을 메모리에서 읽어오기
    * [질문 + 유사도높은 문서 + 이전 대화내용]으로 프롬프트 구성
    * GPT에 질문하고 답변 받기
* Chain
    * 이러한 절차를 코드로 하나하나 엮고, 프롬프트를 구성하는 코드는 상당히 복잡합니다.
    * 랭체인에서 제공하는 Chain 함수를 이용하면 쉽게 구현 가능!  

### (1) Chain 함수로 연결하기
* **ConversationalRetrievalChain**

In [10]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True) # 대화 시작할 때 실행돼야 함

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

### (2) 사용하기

#### 1) 첫번째 질문

In [11]:
# 첫번째 질문
query1 = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query1)
result['answer']

'생성형 AI 도입시 예상되는 보안 위협으로는 악의적인 공격으로 인한 소프트웨어에 악성 코드 추가, 훈련 데이터, 미세 조정, 가중치 오염, 모델을 재학습시켜 사용자 인프라에 침입, 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협이 있을 수 있습니다. 또한 현재 오픈소스 생성형 AI 모델의 안전성을 평가하는 독립적인 기관이 없고, 관련 법률이 시행되기 전까지 보안에 대한 책임과 방어 조치를 제대로 마련하지 않을 경우 보안 위협에 노출될 수 있습니다.'

In [12]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협으로는 악의적인 공격으로 인한 소프트웨어에 악성 코드 추가, 훈련 데이터, 미세 조정, 가중치 오염, 모델을 재학습시켜 사용자 인프라에 침입, 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협이 있을 수 있습니다. 또한 현재 오픈소스 생성형 AI 모델의 안전성을 평가하는 독립적인 기관이 없고, 관련 법률이 시행되기 전까지 보안에 대한 책임과 방어 조치를 제대로 마련하지 않을 경우 보안 위협에 노출될 수 있습니다.')]}

#### 2) 두번째 질문

In [13]:
# 두번째 질문
query2 = "모델을 재학습시면 어떤 문제가 발생되는거야?"
result = qa(query2)
result['answer']

'생성형 AI 분야에서는 전문가 부족으로 인해 보안 취약점에 대응하기 어려울 수 있으며, 오픈소스 소프트웨어를 사용할 때 전문적인 지식이 필요하며, 프로젝트는 악의적인 공격에 노출될 수 있습니다. 해커들은 프로젝트 팀에 침투하여 악성 코드를 추가하거나 모델을 재학습시켜 사용자 인프라에 침입할 수도 있습니다.'

In [14]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협으로는 악의적인 공격으로 인한 소프트웨어에 악성 코드 추가, 훈련 데이터, 미세 조정, 가중치 오염, 모델을 재학습시켜 사용자 인프라에 침입, 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협이 있을 수 있습니다. 또한 현재 오픈소스 생성형 AI 모델의 안전성을 평가하는 독립적인 기관이 없고, 관련 법률이 시행되기 전까지 보안에 대한 책임과 방어 조치를 제대로 마련하지 않을 경우 보안 위협에 노출될 수 있습니다.'),
  HumanMessage(content='모델을 재학습시면 어떤 문제가 발생되는거야?'),
  AIMessage(content='생성형 AI 분야에서는 전문가 부족으로 인해 보안 취약점에 대응하기 어려울 수 있으며, 오픈소스 소프트웨어를 사용할 때 전문적인 지식이 필요하며, 프로젝트는 악의적인 공격에 노출될 수 있습니다. 해커들은 프로젝트 팀에 침투하여 악성 코드를 추가하거나 모델을 재학습시켜 사용자 인프라에 침입할 수도 있습니다.')]}

### (3) 반복문 안에서 질문답변 이어가기

In [16]:
while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)

질문 : 생성형 AI가 뭐야?
--------------------
답변 : 생성형 AI(Generative AI)는 데이터를 분석하고 이를 기반으로 새로운 콘텐츠, 이미지, 음악 또는 텍스트를 생성하는 인공지능 기술입니다. 이는 기존의 데이터나 패턴을 활용하여 새로운 창의적인 결과물을 만들어내는 것을 목표로 합니다.
질문 : 잘 이해가 안 가.
--------------------
답변 : 생성형 AI(Generative AI)는 인공지능 기술의 한 분야로, 새로운 데이터나 콘텐츠를 생성하는 능력을 가진 시스템을 말합니다. 이러한 AI 시스템은 주어진 입력 정보나 지식을 활용하여 새로운 이미지, 음악, 텍스트, 혹은 비디오 등을 자동으로 생성할 수 있습니다. 생성형 AI는 대화형 AI, 이미지 생성, 음악 생성, 자연어 처리 분야 등에서 활발하게 활용되고 있습니다. 그러나 생성형 AI 프로젝트는 보안 측면에서 주의가 필요하며, 악의적인 공격에 취약할 수 있는 점도 있습니다.
질문 : 
--------------------
